In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from common_variables import *
from time_series import *


2025-03-27 22:00:55.165941: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2025-03-27 22:00:55.165958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
epochs = 20
np.random.seed(seed)
tf.random.set_seed(seed)
df_train = pd.read_csv(train_time_series_path, index_col=['time'])
df_val = pd.read_csv(validate_time_series_path, index_col=['time'])


,PC1,PC2,PC3,PC4,open,high,low,close,SMA_50,SMA_200,...,MACD,Signal_Line,Bollinger_Upper,Bollinger_Lower,ATR,K,D,J,MA,Returns
time,,,,,,,,,,,,,,,,,,,,,
2012-12-11 12:00:00+00:00,3.373803,-1.849751,-2.222553,0.357438,1.29927,1.30075,1.29822,1.29984,1.300558,1.288439,...,-0.001358,-0.002335,1.313314,1.287802,0.003124,81.683253,68.443827,108.162105,1.293071,0.000249
2012-12-11 16:00:00+00:00,3.350967,-2.244847,-2.105291,0.358316,1.29986,1.30151,1.29852,1.30040,1.300593,1.288449,...,-0.000859,-0.002040,1.313338,1.287849,0.003137,84.738547,73.875400,106.464840,1.293578,0.000392
2012-12-11 20:00:00+00:00,3.348567,-2.610816,-2.069897,0.040943,1.30034,1.30142,1.30005,1.30113,1.300658,1.288473,...,-0.000401,-0.001712,1.313379,1.287938,0.002785,88.690692,78.813831,108.444414,1.294283,0.000545
2012-12-12 00:00:00+00:00,3.351790,-2.558022,-1.811837,-0.070624,1.30114,1.30126,1.29995,1.30022,1.300717,1.288508,...,-0.000110,-0.001392,1.313401,1.288033,0.002684,87.816833,81.814832,99.820835,1.294768,0.000375
2012-12-12 04:00:00+00:00,3.337806,-2.883005,-1.644090,-0.232802,1.30020,1.30114,1.29958,1.30032,1.300780,1.288537,...,0.000127,-0.001088,1.313424,1.288136,0.002504,87.281519,83.637061,94.570436,1.295529,0.000587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-30 01:00:00+00:00,-1.674972,-0.993963,-0.169435,-0.224064,1.08589,1.08715,1.08546,1.08598,1.083469,1.088975,...,0.000924,0.000628,1.089535,1.077404,0.002955,62.180009,59.681671,67.176684,1.084815,0.000235
2020-04-30 05:00:00+00:00,-1.705656,-1.640552,-0.573257,-0.060831,1.08601,1.08914,1.08535,1.08858,1.083520,1.088859,...,0.001127,0.000728,1.089714,1.077326,0.003119,72.236582,63.866642,88.976462,1.085129,0.000290
2020-04-30 09:00:00+00:00,-1.715922,-1.651083,-0.438163,0.004579,1.08855,1.08882,1.08591,1.08760,1.083529,1.088731,...,0.001196,0.000822,1.089747,1.077311,0.003203,71.657082,66.463455,82.044337,1.085472,0.000316


In [3]:
# Scale the data (important for LSTM)
X_train, y_train, X_scaler, y_scaler = get_train(df_train.iloc[:, :4],df_train['Returns'], window_size)


X shape: (11232, 256, 4), y shape: (11232, 1)


In [4]:
X_val, y_val = get_val_test(df_val.iloc[:, :4],df_val['Returns'], window_size, X_scaler, y_scaler)

X (3808, 256, 4), y (3808, 1)


In [6]:

# Define the LSTM Model
model = Sequential()

# First LSTM layer (Bidirectional) 
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))  
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001)))  
model.add(Dropout(0.2))

# Fully connected layers
model.add(Dense(32, activation='relu'))  
model.add(Dense(1))  

# Compile the model
optimizer = Adam(learning_rate=0.0003)
model.compile(loss="huber", optimizer=optimizer)

# Force the model to build
model.build(input_shape=(None, X_train.shape[1], X_train.shape[2]))

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 256, 256)          136192    
_________________________________________________________________
dropout (Dropout)            (None, 256, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 220,481
Trainable params: 220,481
Non-trainable params: 0
__________________________________________________

2025-03-27 22:00:55.698486: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2025-03-27 22:00:55.706827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-27 22:00:55.708189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.52GHz coreCount: 128 deviceMemorySize: 23.54GiB deviceMemoryBandwidth: 938.86GiB/s
2025-03-27 22:00:55.708236: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2025-03-27 22:00:55.708256: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic libr

In [7]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = epochs, batch_size = batch_size, shuffle=False, verbose = 2)

Epoch 1/20
351/351 - 36s - loss: 0.3174 - val_loss: 0.2316
Epoch 2/20
351/351 - 35s - loss: 0.2037 - val_loss: 0.1682
Epoch 3/20
351/351 - 35s - loss: 0.1623 - val_loss: 0.1403
Epoch 4/20
351/351 - 35s - loss: 0.1370 - val_loss: 0.1221
Epoch 5/20
351/351 - 35s - loss: 0.1210 - val_loss: 0.1103
Epoch 6/20
351/351 - 36s - loss: 0.1063 - val_loss: 0.0956
Epoch 7/20
351/351 - 35s - loss: 0.0949 - val_loss: 0.0900
Epoch 8/20
351/351 - 36s - loss: 0.0854 - val_loss: 0.0744
Epoch 9/20
351/351 - 36s - loss: 0.0790 - val_loss: 0.0718
Epoch 10/20
351/351 - 36s - loss: 0.0750 - val_loss: 0.0642
Epoch 11/20
351/351 - 36s - loss: 0.0706 - val_loss: 0.0595
Epoch 12/20
351/351 - 36s - loss: 0.0667 - val_loss: 0.0607
Epoch 13/20
351/351 - 36s - loss: 0.0642 - val_loss: 0.0593
Epoch 14/20
351/351 - 36s - loss: 0.0618 - val_loss: 0.0525
Epoch 15/20
351/351 - 36s - loss: 0.0616 - val_loss: 0.0509
Epoch 16/20
351/351 - 36s - loss: 0.0575 - val_loss: 0.0553
Epoch 17/20
351/351 - 36s - loss: 0.0563 - val_lo

In [8]:
save_model(model, model_path)
scalers = {
    'X_scaler': X_scaler,
    'y_scaler': y_scaler
}

# Save the dictionary of scalers
joblib.dump(scalers, scaler_path)


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2025-03-27 22:12:52.873410: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/EURUSD, 240_bb79c-32-256-14/assets


['scalers/EURUSD, 240_bb79c-32-256-14.bin']